In [7]:
import pandas as pd
from yahoofinancials import YahooFinancials

In [31]:
import sys
!{sys.executable} -m pip install dataframe_image

  Using cached dataframe_image-0.1.1-py3-none-any.whl (32 kB)
  Using cached aiohttp-3.7.3-cp38-cp38-win_amd64.whl (634 kB)
  Using cached async_timeout-3.0.1-py3-none-any.whl (8.2 kB)
  Using cached multidict-5.1.0-cp38-cp38-win_amd64.whl (48 kB)
  Using cached yarl-1.6.3-cp38-cp38-win_amd64.whl (125 kB)


In [8]:
import dataframe_image as dfi

In [20]:
#stocks = ["PG", "WMT", "PEP", "KO", "MDLZ", "COST", "PM", "MO", "CL", "EL", "KMB", "SYY", "STZ", "WBA", "GIS", "K"]
stocks2 = ["CLF", "OLN", "FLR", "ATI", "TEX", "OKE", "MYRG", "ENLC", "ROAD", "HWM", "MTZ", "KALU", "APOG", "AM"]
stocks = ["AAPL"]

elements = ["symbol", "totalLiab/ebitda", "ebitda/interest", "ltDebt/totalLiab", "currentLiab/totalLiab", "totalLiab/marketCap", "ltDebt/ebitda", "ebitda/totalRevenue", "opIncome/totalRevenue", "netIncome/totalRevenue", "payoutRatio", "cashOpActivities/totalAssets", "capEx/cashOpActivities", "dividendYield", "RevenueDIF", "ebitDIF", "netIncomeDIF", "cashFromOpActivitiesDIF", "dividendsPaidDIF", "totalLiabDIF"]
elements_first = ["totalLiab/ebitda", "ebitda/interest", "ltDebt/totalLiab", "currentLiab/totalLiab", "totalLiab/marketCap", "ltDebt/ebitda", "ebitda/totalRevenue", "opIncome/totalRevenue", "netIncome/totalRevenue", "payoutRatio", "cashOpActivities/totalAssets", "capEx/cashOpActivities", "dividendYield", "RevenueDIF", "ebitDIF", "netIncomeDIF", "cashFromOpActivitiesDIF", "dividendsPaidDIF", "totalLiabDIF", "company"]

In [4]:
#done in branch work

# first dataframe
### only companies with dividendspaid != NaN

In [21]:
data = []

In [10]:
print(data)

[]


In [22]:
for stock in stocks:
    stock1 = YahooFinancials(stocks)
    cashflow = stock1.get_financial_stmts("annual", "cash")
    income = stock1.get_financial_stmts("annual", "income")
    balance = stock1.get_financial_stmts("annual", "balance")
    market_cap = stock1.get_stock_price_data()
    
    print(income)
    
    cash_statement = list(cashflow.keys())[0]
    income_statement = list(income.keys())[0]
    balance_statement = list(balance.keys())[0]
    
    print(income_statement)
    
    dict_i = income[income_statement][stock][-1]
#    print(dict_i)
    d_inc_bef = list(dict_i)[0]
    dict_inc = dict_i.get(d_inc_bef)
    
    dict_c = cashflow[cash_statement][stock][-1]
    d_cash_bef = list(dict_c)[0]
    dict_cash = dict_c.get(d_cash_bef)
    
    dict_b = balance[balance_statement][stock][-1]
    d_bal_bef = list(dict_b)[0]
    dict_bal = dict_b.get(d_bal_bef)
    
    dictionary_before = dict(dict_inc)
    dictionary_before.update(dict_cash)
    dictionary_before.update(dict_bal)
    
    dictionary_c = cashflow[cash_statement][stock][0]
    d_cash_tot = list(dictionary_c)[0]
    
    dictionary_i = income[income_statement][stock][0]
    d_inc_tot = list(dictionary_i)[0]
    
    dictionary_b = balance[balance_statement][stock][0]
    d_bal_tot = list(dictionary_b)[0]
    
    dictionary_m = market_cap[stock]

    d_cash = dictionary_c.get(d_cash_tot)
    d_income = dictionary_i.get(d_inc_tot)
    d_balance = dictionary_b.get(d_bal_tot)
    
    dictionary_total = dict(d_cash)
    dictionary_total.update(d_income)
    dictionary_total.update(d_balance)
    dictionary_total.update(dictionary_m)
    
    try:
        ratios = {"totalLiab/ebitda" : dictionary_total["totalLiab"] / (dictionary_total["ebit"] + dictionary_total["depreciation"]),
                "ebitda/interest" : (dictionary_total["ebit"] - dictionary_total["depreciation"]) / ( - dictionary_total["interestExpense"]),
                "ltDebt/totalLiab" : str((dictionary_total["longTermDebt"] / dictionary_total["totalLiab"]) * 100) + " %",
                "currentLiab/totalLiab" : str((dictionary_total["totalCurrentLiabilities"] / dictionary_total["totalLiab"]) * 100) + " %",
                "totalLiab/marketCap" : dictionary_total["totalLiab"] / dictionary_total["marketCap"],
                "ltDebt/ebitda" : dictionary_total["longTermDebt"] / (dictionary_total["ebit"] + dictionary_total["depreciation"]),
                "ebitda/totalRevenue" : str(((dictionary_total["ebit"] + dictionary_total["depreciation"]) / dictionary_total["totalRevenue"]) * 100) + " %",
                "opIncome/totalRevenue" : str((dictionary_total["operatingIncome"] / dictionary_total["totalRevenue"]) * 100) + " %",
                "netIncome/totalRevenue" : str((dictionary_total["netIncome"] / dictionary_total["totalRevenue"]) * 100) + " %",
                "payoutRatio" : str((- dictionary_total["dividendsPaid"] / dictionary_total["netIncome"]) * 100) + " %",
                "cashOpActivities/totalAssets" : str((dictionary_total["totalCashFromOperatingActivities"] / dictionary_total["totalAssets"]) * 100) + " %",
                "capEx/cashOpActivities" : str((- dictionary_total["capitalExpenditures"] / dictionary_total["totalCashFromOperatingActivities"]) * 100) + " %",
                "dividendYield" : str((- dictionary_total["dividendsPaid"] / dictionary_total["marketCap"]) * 100) + " %",
                "RevenueDIF" : str((dictionary_total["totalRevenue"] / dict_inc["totalRevenue"] - 1) * 100) + " %",
                "ebitDIF" : str((((dictionary_total["ebit"] + dictionary_total["depreciation"]) / (dictionary_before["ebit"] + dictionary_before["depreciation"])) - 1)* 100) + " %",
                "netIncomeDIF" : str(((dictionary_total["netIncome"] / dictionary_before["netIncome"]) - 1) * 100) + " %",
                "cashFromOpActivitiesDIF" : str((dictionary_total["totalCashFromOperatingActivities"] / dictionary_before["totalCashFromOperatingActivities"] - 1) * 100) + " %",
                "dividendsPaidDIF" : str((dictionary_total["dividendsPaid"] / dictionary_before["dividendsPaid"] - 1) * 100) + " %",
                "totalLiabDIF" : str((dictionary_total["totalLiab"] / dictionary_before["totalLiab"] - 1) * 100) + " %",
                "company" : dictionary_m["symbol"]
                }
    except KeyError:
        pass
    
    print(ratios)
    
    dictionary_total.update(ratios)
    
    data.append(ratios)


{'incomeStatementHistory': {'AAPL': [{'2020-09-26': {'researchDevelopment': 18752000000, 'effectOfAccountingCharges': None, 'incomeBeforeTax': 67091000000, 'minorityInterest': None, 'netIncome': 57411000000, 'sellingGeneralAdministrative': 19916000000, 'grossProfit': 104956000000, 'ebit': 66288000000, 'operatingIncome': 66288000000, 'otherOperatingExpenses': None, 'interestExpense': -2873000000, 'extraordinaryItems': None, 'nonRecurring': None, 'otherItems': None, 'incomeTaxExpense': 9680000000, 'totalRevenue': 274515000000, 'totalOperatingExpenses': 208227000000, 'costOfRevenue': 169559000000, 'totalOtherIncomeExpenseNet': 803000000, 'discontinuedOperations': None, 'netIncomeFromContinuingOps': 57411000000, 'netIncomeApplicableToCommonShares': 57411000000}}, {'2019-09-28': {'researchDevelopment': 16217000000, 'effectOfAccountingCharges': None, 'incomeBeforeTax': 65737000000, 'minorityInterest': None, 'netIncome': 55256000000, 'sellingGeneralAdministrative': 18245000000, 'grossProfit':

In [23]:
df = pd.DataFrame(columns=elements_first)
df = df.append(data)
df = df.drop_duplicates(subset=["company"])
df = df[df["company"].notna()]
df = df.iloc[:, [0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]
df = df.set_index("company")
df

,totalLiab/ebitda,ebitda/interest,ltDebt/totalLiab,currentLiab/totalLiab,totalLiab/marketCap,ltDebt/ebitda,ebitda/totalRevenue,opIncome/totalRevenue,netIncome/totalRevenue,payoutRatio,cashOpActivities/totalAssets,capEx/cashOpActivities,dividendYield,RevenueDIF,ebitDIF,netIncomeDIF,cashFromOpActivitiesDIF,dividendsPaidDIF,totalLiabDIF
company,,,,,,,,,,,,,,,,,,,
AAPL,3.342845,19.224504,38.16181845607603 %,40.76287280167396 %,0.132354,1.27569,28.17478097736007 %,24.147314354406863 %,20.913611278072235 %,24.526658654264864 %,24.907992886429877 %,9.059920172546297 %,0.7208223088289382 %,19.753177975343974 %,8.171913679528963 %,18.737978531984865 %,25.611521992993392 %,10.274884485864199 %,7.160797771809402 %


In [33]:
dfi.export(df, "df_extraction.png")

In [34]:
df_styled = df.style.background_gradient()
dfi.export(df_styled, "df_extraction_styled.png")

In [68]:
df_filtered = df.loc[(df["ebitda/interest"]> -5) & (df["ltDebt/ebitda"] > 1)]
df_filtered

,totalLiab/ebitda,ebitda/interest,ltDebt/totalLiab,currentLiab/totalLiab,totalLiab/marketCap,ltDebt/ebitda,ebitda/totalRevenue,opIncome/totalRevenue,netIncome/totalRevenue,payoutRatio,cashOpActivities/totalAssets,capEx/cashOpActivities,dividendYield,RevenueDIF,ebitDIF,netIncomeDIF,cashFromOpActivitiesDIF,dividendsPaidDIF,totalLiabDIF
company,,,,,,,,,,,,,,,,,,,
OLN,7.843258,1.363487,0.492666,0.159845,1.645720,3.864110,0.141277,0.043502,-0.001849,-11.442478,0.067187,0.624656,0.031430,1.100782,1.030687,2.897436,1.023375,0.978804,1.043254
TEX,5.883286,-3.248009,0.514779,0.385455,0.870687,3.028594,0.088374,0.076980,0.012497,0.577206,0.054262,0.628028,0.012080,0.979744,1.723566,-0.308915,0.459825,1.046667,0.649329
OKE,6.546764,-2.903116,0.807866,0.129621,0.794037,5.288908,0.234225,0.187342,0.125790,1.140039,0.089252,1.976778,0.074259,1.139384,1.427625,3.631919,1.438627,2.816123,1.226308
ENLC,5.203937,0.798148,0.863626,0.117133,2.376420,4.494254,0.175805,0.073628,-0.185361,-0.417404,0.106247,0.761065,0.200971,1.416325,1.473830,2.433261,1.488445,2.519957,1.103774
HWM,6.244337,-2.967456,0.378637,0.318438,1.069127,2.364337,0.146209,0.108441,0.033117,0.121277,0.023097,1.443350,0.004703,1.145070,0.902566,-0.499469,4.273684,0.250000,0.869772
KALU,4.075617,-3.910569,0.621734,0.215070,0.486569,2.533951,0.128393,0.095965,0.040948,0.635484,0.152208,0.259148,0.024196,1.137908,0.897921,0.676118,1.402778,1.216049,1.240294
APOG,5.116998,-2.826625,0.347101,0.452223,0.639839,1.776117,0.086233,0.052505,0.044625,0.302258,0.095007,0.479462,0.019558,1.244861,0.758040,0.721693,0.865009,1.275926,1.949220
AM,5.919842,-2.035806,0.921256,0.077110,0.774902,5.453688,0.624212,0.444378,-0.417979,-1.397289,0.099061,0.629817,0.122474,50.141525,3.139602,-36.568222,65.260250,30.991006,442.178028


# second dataframe

### all the companies, but without the ratios including dividendspaid

In [66]:
data1 = []

In [67]:
for stock in stocks:
    stock1 = YahooFinancials(stocks)
    cashflow = stock1.get_financial_stmts("annual", "cash")
    income = stock1.get_financial_stmts("annual", "income")
    balance = stock1.get_financial_stmts("annual", "balance")
    market_cap = stock1.get_stock_price_data()
    
    cash_statement = list(cashflow.keys())[0]
    income_statement = list(income.keys())[0]
    balance_statement = list(balance.keys())[0]
    
    dict_i = income[income_statement][stock][-1]
    d_inc_bef = list(dict_i)[0]
    dict_inc = dict_i.get(d_inc_bef)
    
    dict_c = cashflow[cash_statement][stock][-1]
    d_cash_bef = list(dict_c)[0]
    dict_cash = dict_c.get(d_cash_bef)
    
    dict_b = balance[balance_statement][stock][-1]
    d_bal_bef = list(dict_b)[0]
    dict_bal = dict_b.get(d_bal_bef)
    
    dictionary_before = dict(dict_inc)
    dictionary_before.update(dict_cash)
    dictionary_before.update(dict_bal)
    
    dictionary_c = cashflow[cash_statement][stock][0]
    d_cash_tot = list(dictionary_c)[0]
    
    dictionary_i = income[income_statement][stock][0]
    d_inc_tot = list(dictionary_i)[0]
    
    dictionary_b = balance[balance_statement][stock][0]
    d_bal_tot = list(dictionary_b)[0]
    
    dictionary_m = market_cap[stock]

    d_cash = dictionary_c.get(d_cash_tot)
    d_income = dictionary_i.get(d_inc_tot)
    d_balance = dictionary_b.get(d_bal_tot)
    
    dictionary_total = dict(d_cash)
    dictionary_total.update(d_income)
    dictionary_total.update(d_balance)
    dictionary_total.update(dictionary_m)
    
    ratios = {"totalLiab/ebitda" : dictionary_total["totalLiab"] / (dictionary_total["ebit"] + dictionary_total["depreciation"]),
            "ebitda/interest" : (dictionary_total["ebit"] - dictionary_total["depreciation"]) / dictionary_total["interestExpense"],
            "ltDebt/totalLiab" : dictionary_total["longTermDebt"] / dictionary_total["totalLiab"],
            "currentLiab/totalLiab" : dictionary_total["totalCurrentLiabilities"] / dictionary_total["totalLiab"],
            "totalLiab/marketCap" : dictionary_total["totalLiab"] / dictionary_total["marketCap"],
            "ltDebt/ebitda" : dictionary_total["longTermDebt"] / (dictionary_total["ebit"] + dictionary_total["depreciation"]),
            "ebitda/totalRevenue" : (dictionary_total["ebit"] + dictionary_total["depreciation"]) / dictionary_total["totalRevenue"],
            "opIncome/totalRevenue" : dictionary_total["operatingIncome"] / dictionary_total["totalRevenue"],
            "netIncome/totalRevenue" : dictionary_total["netIncome"] / dictionary_total["totalRevenue"],
#            "payoutRatio" : - dictionary_total["dividendsPaid"] / dictionary_total["netIncome"],
            "cashOpActivities/totalAssets" : dictionary_total["totalCashFromOperatingActivities"] / dictionary_total["totalAssets"],
            "capEx/cashOpActivities" : - dictionary_total["capitalExpenditures"] / dictionary_total["totalCashFromOperatingActivities"],
#            "dividendYield" : - dictionary_total["dividendsPaid"] / dictionary_total["marketCap"],
            "RevenueDIF" : dictionary_total["totalRevenue"] / dict_inc["totalRevenue"],
            "ebitDIF" : (dictionary_total["ebit"] + dictionary_total["depreciation"]) / (dictionary_before["ebit"] + dictionary_before["depreciation"]),
            "netIncomeDIF" : dictionary_total["netIncome"] / dictionary_before["netIncome"],
            "cashFromOpActivitiesDIF" : dictionary_total["totalCashFromOperatingActivities"] / dictionary_before["totalCashFromOperatingActivities"],
#            "dividendsPaidDIF" : dictionary_total["dividendsPaid"] / dictionary_before["dividendsPaid"],
            "totalLiabDIF" : dictionary_total["totalLiab"] / dictionary_before["totalLiab"],
            }

    print(ratios)
    
    dictionary_total.update(ratios)
    
    data1.append(ratios)


OSError: [Errno socket error] [WinError 10065] Se ha intentado una operación de socket en un host no accesible

In [ ]:
df1 = pd.DataFrame(columns=elements)
df1 = df.append(data1)
df1 = df.drop_duplicates(subset=["totalLiab/ebitda"])
df1 = df.iloc[:, [0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]
df1 = df.set_index("symbol")
df1

In [ ]:
df1_filtered = df1.loc[(df["ebitda/interest"]> -5) & (df["ltDebt/ebitda"] > 1)]
df1_filtered

### drafts

In [ ]:
ratios["totalLiab/ebitda"] = dictionary_total["totalLiab"] / (dictionary_total["ebit"] + dictionary_total["depreciation"])

stocks = ["CLF", "OLN", "FLR", "ATI", "TEX", "OKE", "MYRG", "ENLC", "ROAD", "HWM", "MTZ", "KALU", "APOG", "AM"]


elements = ["totalLiab/ebitda", "ebitda/interest", "ltDebt/totalLiab", "currentLiab/totalLiab", "totalLiab/marketCap", "ltDebt/ebitda", "ebitda/totalRevenue", "opIncome/totalRevenue", "netIncome/totalRevenue", "payoutRatio", "cashOpActivities/totalAssets", "capEx/cashOpActivities", "dividendYield", "RevenueDIF", "ebitDIF", "netIncomeDIF", "cashFromOpActivitiesDIF", "dividendsPaidDIF", "totalLiabDIF"]

ratios = {"totalLiab/ebitda" : dictionary_total["totalLiab"] / (dictionary_total["ebit"] + dictionary_total["depreciation"]),
        "ebitda/interest" : (dictionary_total["ebit"] - dictionary_total["depreciation"]) / dictionary_total["interestExpense"],
        "ltDebt/totalLiab" : dictionary_total["longTermDebt"] / dictionary_total["totalLiab"],
        "currentLiab/totalLiab" : dictionary_total["totalCurrentLiabilities"] / dictionary_total["totalLiab"],
        "totalLiab/marketCap" : dictionary_total["totalLiab"] / dictionary_total["marketCap"],
        "ltDebt/ebitda" : dictionary_total["longTermDebt"] / (dictionary_total["ebit"] + dictionary_total["depreciation"]),
        "ebitda/totalRevenue" : (dictionary_total["ebit"] + dictionary_total["depreciation"]) / dictionary_total["totalRevenue"],
        "opIncome/totalRevenue" : dictionary_total["operatingIncome"] / dictionary_total["totalRevenue"],
        "netIncome/totalRevenue" : dictionary_total["netIncome"] / dictionary_total["totalRevenue"],
        "payoutRatio" : - dictionary_total["dividendsPaid"] / dictionary_total["netIncome"],
        "cashOpActivities/totalAssets" : dictionary_total["totalCashFromOperatingActivities"] / dictionary_total["totalAssets"],
        "capEx/cashOpActivities" : - dictionary_total["capitalExpenditures"] / dictionary_total["totalCashFromOperatingActivities"],
        "dividendYield" : - dictionary_total["dividendsPaid"] / dictionary_total["marketCap"],
        "RevenueDIF" : dictionary_total["totalRevenue"] / dict_inc["totalRevenue"],
        "ebitDIF" : (dictionary_total["ebit"] + dictionary_total["depreciation"]) / (dictionary_before["ebit"] + dictionary_before["depreciation"]),
        "netIncomeDIF" : dictionary_total["netIncome"] / dictionary_before["netIncome"],
        "cashFromOpActivitiesDIF" : dictionary_total["totalCashFromOperatingActivities"] / dictionary_before["totalCashFromOperatingActivities"],
        "dividendsPaidDIF" : dictionary_total["dividendsPaid"] / dictionary_before["dividendsPaid"],
        "totalLiabDIF" : dictionary_total["totalLiab"] / dictionary_before["totalLiab"],
        "symbol" : dictionary_m["symbol"]
        } 

try:
    ratios = {"totalLiab/ebitda" : dictionary_total["totalLiab"] / (dictionary_total["ebit"] + dictionary_total["depreciation"]),
                "ebitda/interest" : (dictionary_total["ebit"] - dictionary_total["depreciation"]) / dictionary_total["interestExpense"],
                "ltDebt/totalLiab" : dictionary_total["longTermDebt"] / dictionary_total["totalLiab"],
                "currentLiab/totalLiab" : dictionary_total["totalCurrentLiabilities"] / dictionary_total["totalLiab"],
                "totalLiab/marketCap" : dictionary_total["totalLiab"] / dictionary_total["marketCap"],
                "ltDebt/ebitda" : dictionary_total["longTermDebt"] / (dictionary_total["ebit"] + dictionary_total["depreciation"]),
                "ebitda/totalRevenue" : (dictionary_total["ebit"] + dictionary_total["depreciation"]) / dictionary_total["totalRevenue"],
                "opIncome/totalRevenue" : dictionary_total["operatingIncome"] / dictionary_total["totalRevenue"],
                "netIncome/totalRevenue" : dictionary_total["netIncome"] / dictionary_total["totalRevenue"],
                "payoutRatio" : - dictionary_total["dividendsPaid"] / dictionary_total["netIncome"],
                "cashOpActivities/totalAssets" : dictionary_total["totalCashFromOperatingActivities"] / dictionary_total["totalAssets"],
                "capEx/cashOpActivities" : - dictionary_total["capitalExpenditures"] / dictionary_total["totalCashFromOperatingActivities"],
                "dividendYield" : - dictionary_total["dividendsPaid"] / dictionary_total["marketCap"],
                "RevenueDIF" : dictionary_total["totalRevenue"] / dict_inc["totalRevenue"],
                "ebitDIF" : (dictionary_total["ebit"] + dictionary_total["depreciation"]) / (dictionary_before["ebit"] + dictionary_before["depreciation"]),
                "netIncomeDIF" : dictionary_total["netIncome"] / dictionary_before["netIncome"],
                "cashFromOpActivitiesDIF" : dictionary_total["totalCashFromOperatingActivities"] / dictionary_before["totalCashFromOperatingActivities"],
                "dividendsPaidDIF" : dictionary_total["dividendsPaid"] / dictionary_before["dividendsPaid"],
                "totalLiabDIF" : dictionary_total["totalLiab"] / dictionary_before["totalLiab"],
                }
except:
    pass